# Schrier Lab `contrastive` fork issue demonstration

Here we will use the Mice Protein Dataset example from the paper to illustrate our issue in extending `contrastive`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from matplotlib.colors import ListedColormap
%matplotlib inline
warnings.filterwarnings('ignore')

# for plotting
cmap2 = ListedColormap(['r', 'k'])
cmap4 = ListedColormap(['k', 'r', 'g', 'b'])

### Load the Mice Protein Dataset

In [ ]:
data = np.genfromtxt('datasets/Data_Cortex_Nuclear.csv',delimiter=',',
                     skip_header=1,usecols=range(1,78),filling_values=0)
classes = np.genfromtxt('datasets/Data_Cortex_Nuclear.csv',delimiter=',',
                        skip_header=1,usecols=range(78,81),dtype=None)

### Split by contrastive feature

In [ ]:
target_idx_A = np.where((classes[:,-1]==b'S/C') & (classes[:,-2]==b'Saline') & (classes[:,-3]==b'Control'))[0]
target_idx_B = np.where((classes[:,-1]==b'S/C') & (classes[:,-2]==b'Saline') & (classes[:,-3]==b'Ts65Dn'))[0]

sub_group_labels = len(target_idx_A)*[0] + len(target_idx_B)*[1]
target_idx = np.concatenate((target_idx_A,target_idx_B))                                                                          

target = data[target_idx]
target = (target-np.mean(target,axis=0)) / np.std(target,axis=0) # standardize the dataset

background_idx = np.where((classes[:,-1]==b'C/S') & (classes[:,-2]==b'Saline') & (classes[:,-3]==b'Control'))
# background_idx = np.where((classes[:,-1]==b'C/S') & (classes[:,-2]==b'Saline') & (classes[:,-3]==b'Ts65Dn'))
background = data[background_idx]
background = (background-np.mean(background,axis=0)) / np.std(background,axis=0) # standardize the dataset
labels = len(background)*[0] + len(target)*[1]
data = np.concatenate((background, target))

In [ ]:
from contrastive import CPCA
mdl = CPCA()
projected_data = mdl.fit_transform(target, background, active_labels=sub_group_labels, plot=True)

## Disussion of issue

Above we see some results of CPCA on the Mice Protein dataset. When alpha=789.65, we would expect cPC1 to take account of nearly *all* of the variance in the dataset, and cPC2 to take account of nearly *none* of it. However, based on our extension of the code, this is not what we see: 

In [ ]:
# mdl.bases is where we store information about the cPC basis sets, keyed by the alpha value
print("Result 1:\n")
print(mdl.bases[789.65]['variance_ratio'])

According to our extension of the code, these vectors account for nearly *none* of the variance. That cant be right., 

Something interesting happens if we store *all* of the cPCs instead of just the top two: 

In [ ]:
# rebuild the cPCA model retaining all cPCs
mdl2 = CPCA(n_components=data.shape[1])
projected_data = mdl2.fit_transform(target, background, active_labels=sub_group_labels)

print("Result 2:\n")
mdl2.bases[789.65]['variance_ratio']

It seems like in Result 1, our code pulled the cPCS accounting for the *least* variance rather than the *most*. We aren't sure why this would be, but perhaps our understanding of the variable `eig_idx` is subtly flawed. We hope you can help clear this up and let us know where we may have gone astray. 